In [9]:
#%cd C:/Users/Mathieu/Desktop/Projets/Benter
%cd /home/mathieu/Mindsay/mathieu/Benter-Project

/home/mathieu/Mindsay/mathieu/Benter-Project


# Train 

- predict the best horse in a given race


### TODO
- try to predict a given distribution according to ranks, instead of the first one
- try to predict best among all combinaisons

In [10]:
%matplotlib inline

import datetime as dt
import pandas as pd
import seaborn as sns
import numpy as np
from tqdm import tqdm
import re
from itertools import combinations
import tensorflow as tf
import functools
from scipy.stats import rankdata
import scipy
import json

from utils import import_data
from winning_validation import errors, probability_at_rank, return_against
import winning_validation
from winning_horse_models import sklearn
from winning_horse_models.logistic_regression import LogisticRegressionModel
from winning_horse_models.xgboost import XGBoostWinningModel
from winning_horse_models.lgbm import LGBMWinningModel
from training_procedures import flattened_training

SOURCE = "PMU"

In [3]:
sequential_sgd_regression = LogisticRegressionModel.load_model()

In [4]:
returns = return_against.compute_return_against_odds(source="PMU",same_races_support=True, winning_model=sequential_sgd_regression, verbose=False);
return_against.compute_overall_average(returns)

(0.04917593446047697,
 0.4947343446536749,
 -0.2528736465547953,
 -2.098604165707568)

In [9]:
exact_top_1 = errors.compute_validation_error(source=SOURCE, 
                                                          k=1,
                                                          winning_model=sequential_sgd_regression, 
                                                          validation_method=errors.exact_top_k)
winning_validation.compute_overall_average(exact_top_1)

(0.23681652490886998, 0.09218307006885379, 0.27112191170514377)

In [6]:
kappa_cohen_1 = errors.compute_validation_error(source=SOURCE, 
                                                            k=1,
                                                            winning_model=sequential_sgd_regression, 
                                                            validation_method=winning_validation.kappa_cohen_like)

In [7]:
precision_at_3 = errors.compute_validation_error(source=SOURCE, 
                                                             k=3,
                                                             winning_model=sequential_sgd_regression, 
                                                             validation_method=winning_validation.precision_at_k)

In [13]:
xgboost_winning_model = XGBoostWinningModel.load_model()

In [14]:
exact_top_1 = errors.compute_validation_error(source=SOURCE, k=1,winning_model=xgboost_winning_model, validation_method=errors.exact_top_k)
errors.compute_overall_average(exact_top_1)

(0.20612203527265355, 0.09519562132576526, 0.2711939995945672)

In [15]:
kappa_cohen_1 = errors.compute_validation_error(source=SOURCE, 
                                                            k=1,
                                                            winning_model=xgboost_winning_model, 
                                                            validation_method=errors.kappa_cohen_like)
errors.compute_overall_average(kappa_cohen_1)

(0.12488284525758817, 0.002130053016071174, 0.19769219098749824)

In [16]:
returns = return_against.compute_return_against_odds(source="PMU",same_races_support=True, winning_model=xgboost_winning_model, verbose=False);
return_against.compute_overall_average(returns)

(0.059144163327034954,
 0.2962006542614572,
 -0.31098611121580805,
 -2.0744722288679323)

In [3]:
lgmb_winning_model = LGBMWinningModel.load_model()

In [4]:
exact_top_1 = winning_validation.compute_validation_error(source=SOURCE, k=1,winning_model=lgmb_winning_model, validation_method=winning_validation.exact_top_k)

For races w/ 3 horses, 9 races in val, top 1 in right order: 44.444% (Random: 22.222%, Odds 55.556%)
For races w/ 4 horses, 77 races in val, top 1 in right order: 29.870% (Random: 35.065%, Odds 50.649%)
For races w/ 5 horses, 354 races in val, top 1 in right order: 31.356% (Random: 16.949%, Odds 44.350%)
For races w/ 6 horses, 829 races in val, top 1 in right order: 25.814% (Random: 16.888%, Odds 36.188%)
For races w/ 7 horses, 1489 races in val, top 1 in right order: 26.998% (Random: 14.909%, Odds 37.139%)
For races w/ 8 horses, 1914 races in val, top 1 in right order: 27.691% (Random: 11.755%, Odds 35.005%)
For races w/ 9 horses, 2072 races in val, top 1 in right order: 22.442% (Random: 12.403%, Odds 33.687%)
For races w/ 10 horses, 2457 races in val, top 1 in right order: 20.228% (Random: 10.623%, Odds 30.322%)
For races w/ 11 horses, 2269 races in val, top 1 in right order: 19.392% (Random: 9.079%, Odds 28.735%)
For races w/ 12 horses, 2740 races in val, top 1 in right order: 20.07

In [3]:

for SklearnModel in [
     sklearn.DecisionTreeModel, sklearn.SVCModel, sklearn.KNNModel,
                     sklearn.RandomForestModel,
                     sklearn.GradientBoostingModel, sklearn.GaussianNBModel, sklearn.LDAModel,
                     sklearn.SGDModel
                    ]:
    print(SklearnModel.__name__)
    sklearn_winning_model = SklearnModel.load_model()
    kappa_cohen_1 = winning_validation.compute_validation_error(source=SOURCE, k=1,winning_model=sklearn_winning_model, 
                                                              validation_method=winning_validation.kappa_cohen_like)
    print(winning_validation.compute_overall_average(kappa_cohen_1))
    print()

DecisionTreeModel
(0.030470628793524007, 0.002130053016071174, 0.19769219098749824)
SVCModel
(0.04685053088385765, 0.002130053016071174, 0.19769219098749824)
KNNModel
(0.1837746083693874, 0.002130053016071174, 0.19769219098749824)
RandomForestModel
(0.09921375068151626, 0.002130053016071174, 0.19769219098749824)
GradientBoostingModel
(0.11398277168714965, 0.002130053016071174, 0.19769219098749824)
GaussianNBModel
(-0.0008610628552462091, 0.002130053016071174, 0.19769219098749824)
LDAModel
(0.06514285743523787, 0.002130053016071174, 0.19769219098749824)
SGDModel
(0.05170616861614152, 0.002130053016071174, 0.19769219098749824)


/home/mathieu/.pyenv/versions/3.7.4/envs/venv_benter/lib/python3.7/site-packages/scikit_learn-0.23.1-py3.7-linux-x86_64.egg/sklearn/linear_model/_base.py:327: RuntimeWarning: invalid value encountered in true_divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))
